In [1]:
import pandas as pd

In [2]:
!wget http://www.paccanarolab.org/static_content/disease_similarity/Bundle.tgz -P external_data/

--2019-05-15 20:04:17--  http://www.paccanarolab.org/static_content/disease_similarity/Bundle.tgz
Resolving www.paccanarolab.org (www.paccanarolab.org)... 134.219.149.219
Connecting to www.paccanarolab.org (www.paccanarolab.org)|134.219.149.219|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684214131 (653M) [application/x-gzip]
Saving to: ‘external_data/Bundle.tgz.1’

Bundle.tgz.1        100%[===================>] 652.52M  33.7MB/s    in 19s     

2019-05-15 20:04:36 (35.2 MB/s) - ‘external_data/Bundle.tgz.1’ saved [684214131/684214131]



In [4]:
!tar zxvf external_data/Bundle.tgz --directory external_data/

./Bundle/Data/Competitors/park_similarity
./Bundle/Data/OutputData/
./Bundle/Data/OutputData/disease_similarity
./Bundle/Data/IntermediateData/
./Bundle/Data/IntermediateData/mim2mesh
./Bundle/Data/IntermediateData/pubmed2mesh
./Bundle/Data/IntermediateData/mim2pubmed


In [5]:
omim_annots =[]
with open('external_data/Bundle/Data/IntermediateData/mim2mesh','r') as mim2mesh_file:
    for row in mim2mesh_file:
        line = row.strip().split('\t')
        omim =line[0]
        for i in range(1,len(line)):
            omim_annots.append([omim,line[i]])

In [6]:
mim2mesh_df = pd.DataFrame(omim_annots, columns=['disease','annotation'])

In [7]:
mim2mesh_df.head()

,disease,annotation
0,110000,D000293
1,110000,D000328
2,110000,D000368
3,110000,D003483
4,110000,D003937


In [13]:
from rdflib import Graph, URIRef, Literal, RDF, ConjunctiveGraph
def to_rdf(g, df, column_types, row_uri):
    """
    Parameters
    ----------
    g : input rdflib.Graph  
    df: DataFrame to be converted into RDF Graph
    column_types: dictonary of column and its type, type can be URI or Literal
    row_uri: rdf:type value for row index, should be URI
    Returns
    -------
    g: rdflib.Graph generated from DataFrame object
    """
    
    for (index, series) in df.iterrows():
        g.add((URIRef(index), RDF.type, URIRef(row_uri)) )
        for (column, value) in series.iteritems():
            if column_types[column] == 'URI':
                g.add((URIRef(index), URIRef(column), URIRef(value)))
            else:
                g.add((URIRef(index), URIRef(column), Literal(value)))
                
    return g

In [9]:
mim2mesh_df['disease'] = mim2mesh_df['disease'].map(lambda x: 'http://bio2rdf.org/omim:'+str(x))

In [10]:
mim2mesh_df['annotation'] = mim2mesh_df['annotation'].map(lambda x: 'http://bio2rdf.org/mesh:'+str(x))

In [11]:
mim2mesh_df.head()

,disease,annotation
0,http://bio2rdf.org/omim:110000,http://bio2rdf.org/mesh:D000293
1,http://bio2rdf.org/omim:110000,http://bio2rdf.org/mesh:D000328
2,http://bio2rdf.org/omim:110000,http://bio2rdf.org/mesh:D000368
3,http://bio2rdf.org/omim:110000,http://bio2rdf.org/mesh:D003483
4,http://bio2rdf.org/omim:110000,http://bio2rdf.org/mesh:D003937


In [12]:
mim2mesh_df = mim2mesh_df.set_index('disease', drop=True)

In [15]:
mim2mesh_df.rename(columns={'annotation':'http://bio2rdf.org/openpredict_vocabulary:annotation'},inplace=True)

In [16]:
column_types ={'http://bio2rdf.org/openpredict_vocabulary:annotation':'URI'}
g =  ConjunctiveGraph(identifier = URIRef('http://bio2rdf.org/openpredict_resource:fairworkflows.dataset.openpredict.meshannot.R1'))     
g=  to_rdf(g, mim2mesh_df, column_types, 'http://bio2rdf.org/omim_vocabulary:Predominantly-phenotypes' )

In [17]:
g.serialize('data/rdf/omim_mesh_annotations.nq', format='nquads')